In [2]:
import pandas as pd 
from datetime import datetime


df= pd.read_csv('dataELO.csv')
df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')
df.tail(20)

C:\Users\Cristina Jorge\AppData\Local\Temp\ipykernel_5236\2335523116.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Data'] = pd.to_datetime(df['Data'], infer_datetime_format=True, errors='coerce')


,Comp,Temp,Data,Casa,GC,GF,Fora,Res,PC,PF,J,Home_Elo,Away_Elo,Home_New_Elo,Away_New_Elo,Elo Casa,Elo Fora,Novo Elo Casa,Novo Elo Fora
22572,Série B,2023.0,2023-08-12,Sampaio Corrêa,1.0,1.0,CRB,E,1,1,222.0,1559.97,1600.68,1558.11,1602.54,1559.97,1600.68,1558.11,1602.54
22573,Série C,2023.0,2023-08-13,Paysandu,4.0,2.0,Náutico,VC,3,0,169.0,1536.17,1513.61,1539.84,1509.94,1536.17,1513.61,1539.84,1509.94
22574,Série A,2023.0,2023-08-13,Flamengo,1.0,1.0,São Paulo,E,1,1,181.0,1860.57,1783.90,1852.19,1792.28,1860.57,1783.90,1852.19,1792.28
22575,Série C,2023.0,2023-08-13,São José-RS,3.0,3.0,Figueirense,E,1,1,168.0,1531.63,1537.36,1530.29,1538.70,1531.63,1537.36,1530.29,1538.70
22576,Série C,2023.0,2023-08-13,Manaus,1.0,1.0,Remo,E,1,1,167.0,1503.02,1556.96,1502.27,1557.71,1503.02,1556.96,1502.27,1557.71
22577,Série C,2023.0,2023-08-13,América-RN,2.0,0.0,Pouso Alegre,VC,3,0,166.0,1550.68,1474.72,1554.01,1471.39,1550.68,1474.72,1554.01,1471.39
22578,Série C,2023.0,2023-08-13,CSA,1.0,1.0,Botafogo-PB,E,1,1,165.0,1599.19,1546.82,1596.86,1549.15,1599.19,1546.82,1596.86,1549.15
22579,Série B,2023.0,2023-08-13,Vitória,1.0,0.0,Ceará,VC,3,0,225.0,1606.46,1647.93,1615.65,1638.74,1606.46,1647.93,1615.65,1638.74
22580,Série B,2023.0,2023-08-13,Chapecoense,0.0,1.0,Atlético-GO,VV,0,3,226.0,1498.70,1643.43,1486.83,1655.30,1498.70,1643.43,1486.83,1655.30
22581,Série B,2023.0,2023-08-13,Guarani,0.0,1.0,Juventude,VV,0,3,227.0,1653.42,1602.16,1637.52,1618.06,1653.42,1602.16,1637.52,1618.06


In [7]:
#This function updates the Elo ratings and creates a ranking

timestamp_df = pd.DataFrame(columns=['Timestamp'])
def update_elo(df):
    #Gives different weights to each competition
    k_factor_map = {
                    'Série C': 10, 
                    'Série B': 20, 
                    'CBR': 25, 
                    'Série A': 30, 
                    }
    teams = {}
    elo_ranking = {}
    average_elo = {}
    team_games = {}
    team_victories = {}
    team_losses = {}
    team_goals = {}
    team_draws = {}
    last_competition = {}
    current_year = datetime.now().year
    
    #Creates a list of teams and gives then an initial rating of 1500
    for index, row in df.iterrows():
        home_team, away_team = row['Casa'], row['Fora']
        comp_level = row['Comp']
        match_date = row['Data'].year
        k_factor = k_factor_map.get(comp_level, 50) 
        if home_team not in teams:
            teams[home_team] = 1500
            elo_ranking[home_team] = teams[home_team]
        if away_team not in teams:
            teams[away_team] = 1500
            elo_ranking[away_team] = teams[away_team]
        home_elo, away_elo = teams[home_team], teams[away_team]
        home_score, away_score = row['GC'], row['GF']
        result = row['Res']
        # Only update the last competition if it's not 'CBR' and the match is in the current year
        if comp_level != 'CBR' and match_date == current_year:
            last_competition[home_team] = comp_level
            last_competition[away_team] = comp_level

        #Gives different weights for home and away games
        if result == 'VC':
            home_result, away_result = 1, 0
        elif result == 'VV':
            home_result, away_result = 0, 1
        else:
            home_result, away_result = 0.5, 0.5
        home_expected = 1 / (1 + 10 ** ((away_elo - home_elo - 100) / 400))
        away_expected = 1 / (1 + 10 ** ((home_elo - away_elo + 100) / 400))
        
        #Calculates the number of goals
        if home_team in team_goals:
            team_goals[home_team] += home_score
        else:
            team_goals[home_team] = home_score

        if away_team in team_goals:
            team_goals[away_team] += away_score
        else:
            team_goals[away_team] = away_score
       # Update the number of victories, losses, and draws for each team
        if result == 'VC':
            home_result, away_result = 1, 0
            if home_team in team_victories:
                team_victories[home_team] += 1
            else:
                team_victories[home_team] = 1
            if away_team in team_losses:
                team_losses[away_team] += 1
            else:
                team_losses[away_team] = 1
        elif result == 'VV':
            home_result, away_result = 0, 1
            if home_team in team_losses:
                team_losses[home_team] += 1
            else:
                team_losses[home_team] = 1
            if away_team in team_victories:
                team_victories[away_team] += 1
            else:
                team_victories[away_team] = 1
        else:
            home_result, away_result = 0.5, 0.5
            if home_team in team_draws:
                team_draws[home_team] += 1
            else:
                team_draws[home_team] = 1
            if away_team in team_draws:
                team_draws[away_team] += 1
            else:
                team_draws[away_team] = 1
        # Update the number of games played for each team
        if home_team in team_games:
            team_games[home_team] += 1
        else:
            team_games[home_team] = 1
        if away_team in team_games:
            team_games[away_team] += 1
        else:
            team_games[away_team] = 1
        
        # Calculate the goal difference and adjust the K factor accordingly
        goal_diff = abs(home_score - away_score)
        if goal_diff >= 4:
            k_factor = k_factor + 0.75 + (goal_diff - 4) / 8
        elif goal_diff == 3:
            k_factor = k_factor + 0.75
        elif goal_diff == 2:
            k_factor = k_factor + 0.5
        else:
            k_factor = k_factor
            
        # Calculate the rating difference factor
        rating_diff = abs(home_elo - away_elo)
        if rating_diff > 400:
            rating_diff = 400
        rating_factor = rating_diff / 400 + 1
            
        teams[home_team] = home_elo + rating_factor * k_factor * (home_result - home_expected)
        teams[away_team] = away_elo + rating_factor * k_factor * (away_result - away_expected)
        teams[home_team] = round(teams[home_team], 2)
        teams[away_team] = round(teams[away_team], 2)
        elo_ranking[home_team] = teams[home_team]
        elo_ranking[away_team] = teams[away_team]
        df.at[index, 'Elo Casa'] = home_elo
        df.at[index, 'Elo Fora'] = away_elo
        df.at[index, 'Novo Elo Casa'] = teams[home_team]
        df.at[index, 'Novo Elo Fora'] = teams[away_team]

        
        for team, elo in teams.items():
            if team in average_elo:
                average_elo[team].append(elo)
            else:
                average_elo[team] = [elo]

    # Compute average Elo for each team
    for team, elo_list in average_elo.items():
        average_elo[team] = round(sum(elo_list) / len(elo_list), 2)

    #Createa the Datasets
    elo_df = pd.DataFrame.from_dict(elo_ranking, orient='index', columns=['Elo'])
    elo_df = elo_df.sort_values(by='Elo', ascending=False)
    elo_df.index.name = 'Time'
    elo_df.reset_index(inplace=True)
    #elo_df = elo_df[~elo_df['Time'].str.contains(r'\(ARG\)|\(BOL\)|\(CHI\)|\(COL\)|\(EQU\)|\(MEX\)|\(PAR\)|\(PER\)|\(URU\)|\(VEN\)', regex=True)]
    elo_df['Jogos'] = elo_df['Time'].map(team_games) 
    elo_df = elo_df[elo_df['Jogos'] >= 30]
    elo_df['Rank'] = elo_df['Elo'].rank(ascending=False, method='dense')
    elo_df['Média'] = elo_df['Time'].map(average_elo).round(2)
    elo_df['#'] = elo_df['Média'].rank(ascending=False, method='dense')
    elo_df['Gols'] = elo_df['Time'].map(team_goals)
    elo_df['Vitórias'] = elo_df['Time'].map(team_victories)
    elo_df['Derrotas'] = elo_df['Time'].map(team_losses)
    elo_df['Empates'] = elo_df['Time'].map(team_draws)
    elo_df['Série'] = elo_df.apply(lambda row: last_competition.get(row['Time'], '-'), axis=1) 
    elo_df.replace({'Série A': 'A','Série B': 'B','Série C': 'C',                   
            }, inplace=True)
    timestamp_now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    timestamp_df.loc[len(timestamp_df)] = [timestamp_now]

    
    return df, elo_df

df, elo_df = update_elo(df)
df.to_csv('dataELO.csv', index=False)
elo_df.to_json('dataELOranking.json', orient='records')
timestamp_df.to_json('timestamps.json')



In [9]:
filt = (elo_df['Série'] == "A")
elo_df[filt]

,Time,Elo,Jogos,Rank,Média,#,Gols,Vitórias,Derrotas,Empates,Série
0,Botafogo,1913.20,856,1.0,1666.61,12.0,1164.0,343,287,226,A
1,Palmeiras,1874.13,847,2.0,1733.62,2.0,1305.0,405,231,211,A
2,Flamengo,1852.19,881,3.0,1733.21,3.0,1300.0,406,241,234,A
3,Fluminense,1825.18,877,4.0,1691.08,10.0,1227.0,361,292,224,A
4,Corinthians,1797.37,858,5.0,1715.29,5.0,1150.0,373,243,242,A
5,Atlético-MG,1793.30,870,6.0,1710.59,7.0,1332.0,387,271,212,A
6,São Paulo,1792.28,831,7.0,1734.84,1.0,1201.0,375,227,229,A
7,Red Bull Bragantino,1791.03,627,8.0,1572.08,27.0,810.0,240,225,162,A
8,Internacional,1785.13,841,9.0,1713.31,6.0,1147.0,379,246,216,A
9,Athletico-PR,1778.39,864,10.0,1675.05,11.0,1144.0,346,306,212,A
